In [165]:
import pandas as pd
import random
import cv2
import os
import shutil

In [170]:
# Load CSV file into a pandas DataFrame
df = pd.read_csv(r"C:\Users\BEE Team\Desktop\code\Summarys_labels\summary_df_background.csv")
# Directory of videos
video_dir = r'C:\Users\BEE Team\Desktop\Videos\Daniele_Vids_Lab\LD\Yellow'
df['Video_path'] = df['file'].apply(lambda x: os.path.join(video_dir, x.split('_labels')[0] + '.mp4'))

# Keep track of previously extracted clips
extracted_clips = []

# Create the output directory if it doesn't exist
output_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips_Background"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    

In [175]:
# Keep track of previously extracted clips for each video
extracted_clips = {}

n_clips = 75

for k in range (n_clips):
    # Extract video clips from each row of the dataframe and save them to a file
    row = df.sample()
    
    # Extract the video path, start frame, end frame, and classification from the row
    video_path = row['Video_path'].values[0]
    start_frame = row['start_frame'].values[0]
    end_frame = row['end_frame'].values[0]
    classification = row['classification'].values[0]
    
    # If this is the first time we're extracting clips from this video, initialize the extracted clips list
    if video_path not in extracted_clips:
        extracted_clips[video_path] = []
    
    
    attempts = 0
    # Keep trying to extract a clip until it doesn't overlap with any previously extracted clips from this video
    while True:
        # Choose a random start frame and end frame for the clip
        random_start_frame = int(random.uniform(start_frame, end_frame - 121))
        random_end_frame = random_start_frame + 120

        # Check if this clip overlaps with any previously extracted clips from this video
        overlap = False
        for clip in extracted_clips[video_path]:
            if random_start_frame < clip['end_frame'] and clip['start_frame'] < random_end_frame:
                overlap = True
                break

        # If there is no overlap, extract the clip and add it to the list of extracted clips for this video
        if not overlap:
            video = cv2.VideoCapture(video_path)
            video.set(cv2.CAP_PROP_POS_FRAMES, random_start_frame)
            clip_frames = []
            for i in range(random_start_frame, random_end_frame):
                ret, frame = video.read()
                if ret:
                    clip_frames.append(frame)
                else:
                    break
            clip_path = os.path.join(output_dir, f'{classification}_{k}.mp4')
            clip = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*'XVID'), 60, (frame.shape[1], frame.shape[0]))
            for frame in clip_frames:
                clip.write(frame)
            clip.release()
            extracted_clips[video_path].append({
                'start_frame': random_start_frame,
                'end_frame': random_end_frame
            })
            break
        attempts += 1
        if attempts >= 10:
            # Break out of the loop after 10 failed attempts
            break

In [179]:
# set the root directory of your videos
root_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips_Background"

# set the directories where you want to move the videos to
train_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\train"
test_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test"
val_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\val"


# set the split ratios
train_ratio = 0.6
test_ratio = 0.2
val_ratio = 0.2

         
# get the list of videos in the directory
video_list = [f for f in os.listdir(root_dir) if f.endswith(".mp4")]

# calculate the number of videos in each split
num_videos = len(video_list)
num_train = int(0.6 * num_videos)
num_test = int(0.2 * num_videos)
num_val = num_videos - num_train - num_test

# shuffle the video list
random.shuffle(video_list)

# move the videos to the appropriate split directory
for i, video in enumerate(video_list):

    if i < num_train:
        dest_dir = os.path.join(train_dir, video.split("_")[0])
    elif i < num_train + num_test:
        dest_dir = os.path.join(test_dir, video.split("_")[0])
    else:
        dest_dir = os.path.join(val_dir, video.split("_")[0])

    os.makedirs(dest_dir, exist_ok=True)
    src_file = os.path.join(root_dir, video)
    dest_file = os.path.join(dest_dir, video)
    shutil.move(src_file, dest_file)

In [180]:
# Run only to delete folders

#shutil.rmtree(r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips_Background")